In [1]:
# IR HW5 Simple Mixture Model MAP@5000
path = './ntust-ir-2020_hw5_new/'
#path = '../HW2-BM25/ntust-ir-2020/'
import numpy as np
import time
import random
from tqdm import tqdm
import numba as nb
from numba import njit
from numba.core import types
from numba.typed import List
from collections import Counter
from math import log

In [2]:
# data of doc and query(List of lists)
query = List()
doc = []
# length of each doc
doc_len = []
# name of doc and query
query_list = []
doc_list = []
# vocabulary
vocabulary = set()
# vocabulary counting(TF)
TF = []
# BackGround Counting
bg_tf = Counter()
# BM25 query vocabulary
bm25_vocabulary = set()
# get doc files and query files
with open(path + 'query_list.txt', 'r') as fq:
    line = fq.read().splitlines()
    for l in line:
        query_list.append(l)
        f_temp = open(path + 'queries/{}.txt'.format(l))
        temp_query = f_temp.read().split()
        for v in temp_query:
            bm25_vocabulary.add(v)
            vocabulary.add(v)
        query.append(List(temp_query))
        f_temp.close()
with open(path + 'doc_list.txt', 'r') as fd:
    line = fd.read().splitlines()
    for l in tqdm(line):
        doc_list.append(l)
        f_temp = open(path + 'docs/{}.txt'.format(l))
        temp_doc = f_temp.read().split()
        doc.append(temp_doc)
        TF.append(Counter(temp_doc))
        bg_tf.update(temp_doc)
        doc_len.append(len(temp_doc))
        for w in temp_doc:
            vocabulary.add(w)
        f_temp.close()

100%|██████████| 30000/30000 [02:38<00:00, 189.09it/s]


In [3]:
# BackGroud variable
bg_len = np.sum(doc_len)
# All doc, query and vocabulary length
d_len = len(doc)
q_len = len(query)
w_len = len(vocabulary)
# vocabulary to list
vocabulary = list(vocabulary)
bm25_vocabulary =list(bm25_vocabulary)
bm25_len = len(bm25_vocabulary)

In [4]:
# Build TF_key and TF_val
TF_key = List()
TF_val = List()
for d in tqdm(range(d_len)):
    TF_key.append(List([k for k in TF[d].keys()]))
    TF_val.append(List([v for v in TF[d].values()]))
# TF_k = List()
# TF_v = List()
# for d in range(d_len):
#     TF_k.append(TF_key[d])
#     TF_v.append(TF_val[d])

100%|██████████| 30000/30000 [00:27<00:00, 1077.21it/s]


In [5]:
# Probability BackGround Model
P_bg = np.zeros(w_len)
for w in tqdm(range(w_len)):
    P_bg[w] = bg_tf[vocabulary[w]] / bg_len

100%|██████████| 154240/154240 [00:00<00:00, 908514.35it/s]


In [6]:
# IDF
@njit
def IDF(idf, tf_key, bm25_voc, dl, bm25l):
    for d in range(dl):
        for v in range(bm25l):
            if(bm25_voc[v] in tf_key[d]):
                idf[v] += 1
    return idf

In [7]:
# BM25 model variable
bm_sim = np.zeros([q_len, d_len], dtype=float) # 150 x 30000
K1 = 0.8
b = 0.7
N = d_len
avg_dl = np.average(doc_len)

In [8]:
# BM25 IDF
idf_doc = np.zeros(len(bm25_vocabulary))
time_idf = time.time()
idf_doc = IDF(idf_doc, TF_key, bm25_vocabulary, d_len, bm25_len)
print('time: {:.5f} mins\n'.format((time.time() - time_idf) / 60))

time: 1.22684 mins



In [9]:
# B25 Model
@njit
def BM25(sim, k1, b_, N_, avg, dl_list, dl, ql, tf_key, tf_val, Query, idf_, bm_voc):
    for q in range(ql):
        for d in range(dl):
            sum = 0.0
            for w in Query[q]:
                sum_tmp = 1.0
                if(w in tf_key[d]):
                    w_tf = tf_val[d][tf_key[d].index(w)]
                    ni_loc = bm_voc.index(w)
                    ni = idf_[ni_loc]
                    sum_tmp = (k1 + 1) * w_tf / (k1 * ((1 - b) + b * dl_list[d] / avg) + w_tf)
                    sum_tmp *= log((N_ - ni + 0.5) / (ni + 0.5))
                sum += sum_tmp
            sim[q][d] = sum
    return sim

In [10]:
time_bm = time.time()
bm_sim = BM25(bm_sim, K1, b, N, avg_dl, doc_len, d_len, q_len, TF_key, TF_val, query, idf_doc, bm25_vocabulary)
print('time: {:.2f} mins\n'.format((time.time() - time_bm) / 60))

time: 1.93 mins



In [11]:
# SMM variable
SMM_1 = int(500)
Rq_1 = np.zeros([q_len, SMM_1], dtype=np.int64)
for q in range(q_len):
    arg = np.argsort(-bm_sim[q])[:SMM_1]
    for s in range(SMM_1):
        Rq_1[q][s] = arg[s]

In [12]:
# Simple Mixture Model initialization
PSMM_temp = np.zeros([q_len, w_len])
P_Tsmm_temp = np.zeros([q_len, w_len])
# Random
for q in tqdm(range(q_len)):
    r_sum = 0
    for w in range(w_len):
        p_temp = random.uniform(1.0, 10.0)
        PSMM_temp[q][w] = p_temp
        r_sum += p_temp
    for w in range(w_len):
        PSMM_temp[q][w] /= r_sum

100%|██████████| 150/150 [00:25<00:00,  5.92it/s]


In [18]:
# SMM E_step
@njit
def E_step(tsmm, alpha_, psmm, p_bg, wl, ql):
    for w in range(wl):
        mol = (1 - alpha_) * psmm[w]
        den = mol + alpha_ * p_bg[w]
        tsmm[w] = mol / den
    return tsmm

In [19]:
# Relevance Doc matrix
@njit
def BRD(r_w, tf_key, tf_val, rel_len, wl, rel_doc, Vocabulary):
    for r in range(rel_len):
        t = rel_doc[r]
        for w in range(wl):
            if(Vocabulary[w] in tf_key[t]):
                loc = tf_key[t].index(Vocabulary[w])
                r_w[r][w] = tf_val[t][loc]
    return r_w

In [20]:
# SMM M_step
@njit
def M_step(psmm, r_w, Vocabulary, wl, ql, tsmm, tf_key, tf_val, rel_doc):
    den = np.sum(np.dot(r_w, np.transpose(tsmm)))
    for w in range(wl):
        mol = 0.0
        for d in rel_doc:
            if(Vocabulary[w] in tf_key[d]):
                mol += tf_val[d][tf_key[d].index(Vocabulary[w])] * tsmm[w]
        psmm[w] = mol / den
    return psmm

In [21]:
iter = 30
alpha = 0.7
P_Tsmm_1 = np.copy(P_Tsmm_temp)
PSMM_1 = np.copy(PSMM_temp)
for q in range(q_len):
    for i in range(iter):
        print("\nE_step_{}:".format(i+1))
        P_Tsmm_1[q] = E_step(P_Tsmm_1[q], alpha, PSMM_temp[q], P_bg, w_len, q_len)
        print(P_Tsmm_1[q])
        Rel_temp = np.zeros([SMM_1, w_len])
        print("BRD:\n")
        Rel_temp = BRD(Rel_temp, TF_key, TF_val, SMM_1, w_len, Rq_1[q], vocabulary)
        print("\nM_step_{}:\n".format(i+1))
        time_smm = time.time()
        PSMM_1[q] = M_step(PSMM_1[q], Rel_temp, vocabulary, w_len, q_len, P_Tsmm_1[q], TF_key, TF_val, Rq_1[q])
        print("SMM out {}\n".format((time.time() - time_smm) / 60))


E_step_1:
[0.92133031 0.02402542 0.87818925 ... 0.96266914 0.82527957 0.92974519]
BRD:



In [ ]:
# KL Divergence
kl_div = np.zeros([q_len, d_len], dtype=float)

In [ ]:
# KL variable
kl_alpha = 0.4
kl_beta = 0.4
kl_lamda = 0.8

In [ ]:
@jit(nopython=True)
def KL_D(KL, alpha_, beta_, psmm, dl, ql, wl, lamda, p_bg, Vocabulary):
    for q in range(ql):
        for d in range(dl):
            sum = 0
            for w in range(wl):
                term_1 = alpha_ / wl
                term_2 = beta_ * psmm[w]
                term_3 = (1 - alpha_ - beta_) * p_bg[w]
                term_4 = log(lamda * (Doc[d].count(Vocabulary[w]) / len(Doc[d])) + (1 - lamda) * p_bg[w])
                sum += (term_1 + term_2 + term_3) * term_4
            KL[q][d] = -sum
    return KL

In [ ]:
time_kl = time.time()
kl_div = KL_D(kl_div, kl_alpha, kl_beta, PSMM_1, d_len, q_len, w_len, kl_lamda, P_bg, vocabulary, doc)
print('time: {:.2f} mins\n'.format((time.time() - time_kl) / 60))